# PolicyFlux — Advanced Actors
Porównujemy wpływ lobby, biczów partyjnych, Speakera i Prezydenta na wyniki głosowań.

In [ ]:
from policyflux.integration import AdvancedActorsConfig, IntegrationConfig, LayerConfig, build_engine
from policyflux.utils.reports import craft_a_bar

SEED = 20260124
NUM_ACTORS = 100
POLICY_DIM = 4
ITERATIONS = 250


def run_actor_scenario(title: str, actors: AdvancedActorsConfig) -> dict:
    config = IntegrationConfig(
        num_actors=NUM_ACTORS,
        policy_dim=POLICY_DIM,
        iterations=ITERATIONS,
        seed=SEED,
        description=title,
        layer_config=LayerConfig(
            public_support=0.52,
            lobbying_intensity=0.18,
            media_pressure=0.12,
            party_line_support=0.58,
            party_discipline_strength=0.40,
        ),
        actors_config=actors,
        aggregation_strategy="average",
    )

    engine = build_engine(config)
    engine.run_simulation()

    total = len(engine.congress_model.congressmen)
    avg_votes_for = sum(engine.results) / len(engine.results)
    pass_rate = sum(1 for v in engine.results if v > total / 2) / len(engine.results)

    print(engine)
    print(f"Pass rate: {pass_rate:.1%}")

    return {
        "title": title,
        "avg_votes_for": avg_votes_for,
        "avg_votes_against": total - avg_votes_for,
        "pass_rate": pass_rate,
    }

In [ ]:
scenarios = [
    run_actor_scenario(
        "Niski wpływ aktorów",
        AdvancedActorsConfig(
            n_lobbyists=1,
            lobbyist_strength=0.2,
            lobbyist_stance=0.3,
            n_whips=1,
            whip_discipline_strength=0.3,
            whip_party_line_support=0.45,
            speaker_agenda_support=0.48,
            president_approval_rating=0.45,
        ),
    ),
    run_actor_scenario(
        "Silne lobby",
        AdvancedActorsConfig(
            n_lobbyists=6,
            lobbyist_strength=0.75,
            lobbyist_stance=1.0,
            n_whips=1,
            whip_discipline_strength=0.35,
            whip_party_line_support=0.50,
            speaker_agenda_support=0.50,
            president_approval_rating=0.50,
        ),
    ),
    run_actor_scenario(
        "Żelazna dyscyplina",
        AdvancedActorsConfig(
            n_lobbyists=2,
            lobbyist_strength=0.30,
            lobbyist_stance=0.4,
            n_whips=4,
            whip_discipline_strength=0.85,
            whip_party_line_support=0.80,
            speaker_agenda_support=0.62,
            president_approval_rating=0.58,
        ),
    ),
]

scenarios

In [ ]:
craft_a_bar(
    data=[s["pass_rate"] * 100 for s in scenarios],
    labels=[s["title"] for s in scenarios],
    title="Wpływ aktorów na skuteczność ustaw",
    xlabel="Scenariusz",
    ylabel="Pass rate [%]",
)